In [1]:
import datetime
import numpy as np
import pandas as pd
import pandas_ta as ta
import matplotlib.pyplot as plt
import mplfinance as mpf
from dateutil.parser import parse
from dataset import Dataset
from portfolio import Portfolio

In [2]:
std_multiple = 3

In [3]:
tf_hours = 1

start_date = parse('2023-01-01')
end_date = datetime.datetime.now() + datetime.timedelta(days=1)
days = (end_date - start_date).days

data = Dataset().get_data(ticker='BTCUSDT', interval=f'{tf_hours}h', start_date=start_date.strftime('%Y-%m-%d'),
                          end_date=end_date.strftime('%Y-%m-%d'))
data.shape

(5888, 9)

In [4]:
data = data[['open', 'high', 'low', 'close']]

In [5]:
data['candle_size'] = data['high'] - data['low']
data['candle_size_mean'] = data['candle_size'].rolling(14).mean()
data['candle_size_std'] = data['candle_size'].rolling(14).std()
data['thr'] = data['candle_size_mean'] + std_multiple * data['candle_size_std']
data['range_high'] = data['low'] + data['thr']
data['range_low'] = data['high'] - data['thr']
data['dev_high'] = (data['close'] < data['range_high']) & (data['high'] >= data['range_high'])
data['dev_low'] = (data['close'] > data['range_low']) & (data['low'] <= data['range_low'])
data['ema'] = data.ta.ema(21)

In [6]:
data.tail()

,open,high,low,close,candle_size,candle_size_mean,candle_size_std,thr,range_high,range_low,dev_high,dev_low,ema
time,,,,,,,,,,,,,
2023-09-03 05:00:00,25863.91,25880.05,25863.90,25874.99,16.15,49.520000,29.525539,138.096616,26001.996616,25741.953384,False,False,25852.888519
2023-09-03 06:00:00,25874.98,25905.00,25873.50,25900.06,31.50,43.091429,21.299829,106.990915,25980.490915,25798.009085,False,False,25857.176836
2023-09-03 07:00:00,25900.06,25946.59,25892.51,25910.00,54.08,40.241429,15.969351,88.149481,25980.659481,25858.440519,False,False,25861.978942
2023-09-03 08:00:00,25909.99,25956.24,25899.96,25956.22,56.28,39.913571,15.555002,86.578577,25986.538577,25869.661423,False,False,25870.546310
2023-09-03 09:00:00,25956.23,25956.30,25894.08,25907.88,62.22,39.954286,15.616895,86.804971,25980.884971,25869.495029,False,False,25873.940282


In [7]:
data[data['dev_high']].shape, data[data['dev_low']].shape

((101, 13), (100, 13))

In [8]:
returns = []
for idx in data[data['dev_high']].index:
    df = data.loc[idx:]
    op = data.loc[idx]['close']
    cl = df.loc[df[df['low'] <= df['ema']].index[0]]['ema']
    ret = (op - cl) / op
    returns.append(ret)
returns = np.array(returns)
result = np.prod(1 + returns)
result

0.5288254568403141

In [9]:
returns = []
for idx in data[data['dev_low']].index:
    df = data.loc[idx:]
    op = data.loc[idx]['close']
    cl = df.loc[df[df['high'] >= df['ema']].index[0]]['ema']
    ret = -(op - cl) / op
    returns.append(ret)
returns = np.array(returns)
result = np.prod(1 + returns)
result

0.5022461604412168

In [19]:
%%time

start_cash = 1000
risk = 0.01
sl_atr_ratio = 2
min_rr = 1

pf = Portfolio(start_cash=start_cash, max_pos_pct=1, max_leverage=100, maker_fee=0.0002, taker_fee=0.00055)

atr = data.ta.atr()


for idx, val in data.iterrows():
    tp = val['ema']
    if pf.trade.loc[0, 'position'] == 'flat' and val['dev_high']:
        entry = val['close']
        sl = val['high'] + sl_atr_ratio * atr.loc[idx]
        rr = np.abs((entry - tp) / (sl - entry))
        if rr > min_rr:
            pf.open_trade(entry, sl, tp, date=idx, risk=risk)
    elif pf.trade.loc[0, 'position'] == 'flat' and val['dev_low']:
        entry = val['close']
        sl = val['low'] - sl_atr_ratio * atr.loc[idx]
        tp = val['ema']
        rr = np.abs((entry - tp) / (sl - entry))
        if rr > min_rr:
            pf.open_trade(entry, sl, tp, date=idx, risk=risk)    
    elif pf.trade.loc[0, 'position'] == 'short' and val['low'] <= tp:
        pf.take_profit(tp, date=idx)
    elif pf.trade.loc[0, 'position'] == 'short' and val['high'] >= sl:
        pf.stop_loss(sl, date=idx)
    elif pf.trade.loc[0, 'position'] == 'long' and val['high'] >= tp:
        pf.take_profit(tp, date=idx)
    elif pf.trade.loc[0, 'position'] == 'long' and val['low'] <= sl:
        pf.stop_loss(sl, date=idx)

CPU times: total: 750 ms
Wall time: 759 ms


In [20]:
pf.trade_ts

,position,size,open_date,close_date,entry,sl,tp,result,balance
0,short,735.130762,2023-01-12 00:00:00,2023-01-12 01:00:00,17943.26,18187.342562,17495.347082,lose,989.191356
1,short,221.42295,2023-01-14 01:00:00,2023-01-14 11:00:00,20891.67,21824.990569,20258.162773,win,995.739594
2,short,466.122349,2023-01-20 21:00:00,2023-01-21 01:00:00,22288.37,22764.498478,21240.38314,lose,985.269463
3,short,435.622006,2023-01-25 22:00:00,2023-01-26 07:00:00,23592.55,24126.155254,22981.371067,win,996.227792
4,short,667.011018,2023-02-14 15:00:00,2023-02-15 13:00:00,22247.39,22579.670392,21784.764065,lose,985.531802
5,short,375.848796,2023-02-15 21:00:00,2023-02-16 01:00:00,24145.04,24778.159091,22676.490368,lose,975.26305
6,short,553.1917,2023-03-12 18:00:00,2023-03-12 19:00:00,20785.52,21151.963489,20390.025294,lose,964.901909
7,short,285.407246,2023-03-13 15:00:00,2023-03-13 16:00:00,23484.63,24278.595978,22023.017991,lose,954.938942
8,short,251.550577,2023-03-14 13:00:00,2023-03-14 20:00:00,25832.57,26813.228736,24834.324586,win,964.470922
9,short,638.829454,2023-04-10 17:00:00,2023-04-10 23:00:00,29036.85,29475.233004,28326.063755,lose,954.123501


In [42]:
t = 1
for idx in data.index:
#     print(f'{t} / {len(data)}')
    t += 1
    if data.loc[idx]['position'] == 0 and data.loc[idx]['close'] >= data.loc[idx]['open']:
        thr = data.loc[idx]['low'] + data.loc[idx]['thr']
        if data.loc[idx]['high'] >= thr:
            data.loc[idx:]['position'] = -1
    elif data.loc[idx]['position'] == 0 and data.loc[idx]['close'] <= data.loc[idx]['open']:
        thr = data.loc[idx]['high'] - data.loc[idx]['thr']
        if data.loc[idx]['low'] <= thr:
            data.loc[idx:]['position'] = 1
    elif data.loc[idx]['position'] == -1 and data.loc[idx]['low'] <= data.loc[idx]['ema']:
        data.loc[idx:]['position'] = 0
    elif data.loc[idx]['position'] == 1 and data.loc[idx]['high'] >= data.loc[idx]['ema']:
        data.loc[idx:]['position'] = 0

C:\Users\aeali\AppData\Local\Temp\ipykernel_6952\3050334298.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[idx:]['position'] = -1
C:\Users\aeali\AppData\Local\Temp\ipykernel_6952\3050334298.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[idx:]['position'] = 1


sell
buy
sell
sell
sell
sell
sell
buy
sell
buy
buy
sell
buy
sell
sell
sell
buy
sell
buy
buy
buy
sell
sell
sell
sell
sell
sell
buy
sell
sell
buy
sell
sell
sell
buy
sell
sell
sell
sell
sell
buy
sell
buy
sell
sell
sell
sell
buy
buy
sell
buy
buy
sell
sell
sell
buy
sell
buy
sell
sell
sell
buy
buy
sell
buy
sell
sell
buy
buy
buy
buy
buy
sell
buy
buy
sell
buy
buy
buy
sell
buy
sell
sell
buy
sell
buy
buy
buy
buy
sell
buy
sell
buy
buy
buy
buy
sell
buy
buy
sell
sell
buy
sell
sell


In [40]:
data[data['position'] == -1]

,open,high,low,close,candle_size,candle_size_mean,candle_size_std,thr,ema,position
time,,,,,,,,,,


In [ ]:
t = 1
for idx, candle in data_high_vol.iterrows():
    print(f'{t} / {data_high_vol.shape[0]}')
    t += 1
    last_sizes = list(data.loc[:idx].iloc[-14:-1]['candle_size'])
    entry_range = get_thr(last_sizes, std_multiple) * 1.5
    if entry_range > candle['candle_size']:
        continue
    
    start_date = idx - datetime.timedelta(minutes=tf_hours * 60)
    end_date = idx - datetime.timedelta(minutes=1)
    data_1m = Dataset().get_data(ticker='BTCUSDT', interval='1m', start_date=start_date.strftime('%Y-%m-%d %H:%M:%S'),
                                 end_date=end_date.strftime('%Y-%m-%d %H:%M:%S'))
    for i in data_1m.index:
        data_1m.loc[i, 'roll_high'] = data_1m.loc[:i]['high'].max()
        data_1m.loc[i, 'roll_low'] = data_1m.loc[:i]['low'].min()
    data_1m['roll_candle_size'] = data_1m['roll_high'] - data_1m['roll_low']
    rough_price = data_1m[data_1m['roll_candle_size'] > entry_range].iloc[0]['close']
    
    if rough_price > candle['open']:
        entry = candle['low'] + entry_range
        exit = candle['close']
        data_high_vol.loc[idx, 'entry'] = entry
        data_high_vol.loc[idx, 'exit'] = exit
        data_high_vol.loc[idx, 'return_pct'] = (entry - exit) / entry
        data_high_vol.loc[idx, 'darwdown'] = (candle['high'] - entry) / entry
    elif rough_price < candle['open']:
        entry = candle['high'] - entry_range
        exit = candle['close']
        data_high_vol.loc[idx, 'entry'] = entry
        data_high_vol.loc[idx, 'exit'] = exit
        data_high_vol.loc[idx, 'return_pct'] = (exit - entry) / entry
        data_high_vol.loc[idx, 'darwdown'] = (entry - candle['low']) / entry
    else:
        data_high_vol.loc[idx, 'entry'] = np.nan
        data_high_vol.loc[idx, 'exit'] = np.nan
        data_high_vol.loc[idx, 'return_pct'] = 0
        data_high_vol.loc[idx, 'darwdown'] = 0

In [ ]:
# t = 1
# for idx, candle in data_high_vol.iterrows():
#     print(f'{t} / {data_high_vol.shape[0]}')
#     t += 1
#     entry_range = candle['candle_size_mean'] + std_multiple * candle['candle_size_std']
    
#     start_date = idx - datetime.timedelta(minutes=tf_hours * 60)
#     end_date = idx - datetime.timedelta(minutes=1)
#     data_1m = Dataset().get_data(ticker='BTCUSDT', interval='1m', start_date=start_date.strftime('%Y-%m-%d %H:%M:%S'),
#                                  end_date=end_date.strftime('%Y-%m-%d %H:%M:%S'))
#     for i in data_1m.index:
#         data_1m.loc[i, 'roll_high'] = data_1m.loc[:i]['high'].max()
#         data_1m.loc[i, 'roll_low'] = data_1m.loc[:i]['low'].min()
#     data_1m['roll_candle_size'] = data_1m['roll_high'] - data_1m['roll_low']
#     rough_price = data_1m[data_1m['roll_candle_size'] > entry_range].iloc[0]['close']
    
#     if rough_price > candle['open']:
#         entry = candle['low'] + entry_range
#         exit = candle['close']
#         data_high_vol.loc[idx, 'entry'] = entry
#         data_high_vol.loc[idx, 'exit'] = exit
#         data_high_vol.loc[idx, 'return_pct'] = (entry - exit) / entry
#         data_high_vol.loc[idx, 'darwdown'] = (candle['high'] - entry) / entry
#     elif rough_price < candle['open']:
#         entry = candle['high'] - entry_range
#         exit = candle['close']
#         data_high_vol.loc[idx, 'entry'] = entry
#         data_high_vol.loc[idx, 'exit'] = exit
#         data_high_vol.loc[idx, 'return_pct'] = (exit - entry) / entry
#         data_high_vol.loc[idx, 'darwdown'] = (entry - candle['low']) / entry
#     else:
#         data_high_vol.loc[idx, 'entry'] = np.nan
#         data_high_vol.loc[idx, 'exit'] = np.nan
#         data_high_vol.loc[idx, 'return_pct'] = 0
#         data_high_vol.loc[idx, 'darwdown'] = 0

In [ ]:
data_high_vol = data_high_vol.dropna()

In [ ]:
data_high_vol['darwdown'].max()

In [ ]:
# for idx, candle in data_high_vol.iterrows():
#     start_date = idx - datetime.timedelta(minutes=tf_hours * 60 - 1)
#     end_date = idx
#     data_1m = Dataset().get_data(ticker='BTCUSDT', interval='1m', start_date=start_date.strftime('%Y-%m-%d %H:%M:%S'),
#                                  end_date=end_date.strftime('%Y-%m-%d %H:%M:%S'))
#     high_idx = data_1m.index[data_1m['high'].argmax()]
#     low_idx = data_1m.index[data_1m['low'].argmin()]
#     if high_idx > low_idx:
#         entry = candle['low'] + candle['candle_size_mean'] + std_multiple * candle['candle_size_std']
#         exit = candle['close']
#         data_high_vol.loc[idx, 'entry'] = entry
#         data_high_vol.loc[idx, 'exit'] = exit
#         data_high_vol.loc[idx, 'return_pct'] = (entry - exit) / entry
#         data_high_vol.loc[idx, 'darwdown'] = (candle['high'] - entry) / entry
#     elif high_idx < low_idx:
#         entry = candle['high'] - (candle['candle_size_mean'] + std_multiple * candle['candle_size_std'])
#         exit = candle['close']
#         data_high_vol.loc[idx, 'entry'] = entry
#         data_high_vol.loc[idx, 'exit'] = exit
#         data_high_vol.loc[idx, 'return_pct'] = (exit - entry) / entry
#         data_high_vol.loc[idx, 'darwdown'] = (entry - candle['low']) / entry
#     else:
#         data_high_vol.loc[idx, 'entry'] = np.nan
#         data_high_vol.loc[idx, 'exit'] = np.nan
#         data_high_vol.loc[idx, 'return_pct'] = 0
#         data_high_vol.loc[idx, 'darwdown'] = 0

In [ ]:
winrate = data_high_vol[data_high_vol['return_pct'] > 0].shape[0] / data_high_vol.shape[0]
winrate

In [ ]:
(data_high_vol['return_pct'].max(), data_high_vol['return_pct'].min(),
data_high_vol['return_pct'].mean(), data_high_vol['return_pct'].std())

In [ ]:
(data_high_vol['darwdown'].max(), data_high_vol['darwdown'].min(),
data_high_vol['darwdown'].mean(), data_high_vol['darwdown'].std())

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(data_high_vol['return_pct'], bins=50)
plt.title('Returns')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(data_high_vol['darwdown'], bins=50)
plt.title('Drawdowns')
plt.show()

In [ ]:
max_drawdown = 0.1

In [ ]:
data_high_vol['darwdown'].mean() + 5 * data_high_vol['darwdown'].std()

In [ ]:
data_high_vol['return_pct_tot'] = 1 + data_high_vol['return_pct']

In [ ]:
monthly_returns = data_high_vol['return_pct_tot'].resample('M').prod() - 1
monthly_returns

In [ ]:
monthly_returns.mean(), monthly_returns.std()

In [ ]:
sharpe_ratio = (monthly_returns.mean() - 0.004) / monthly_returns.std()
sharpe_ratio

In [ ]:
leverage = max_drawdown / (data_high_vol['darwdown'].mean() + 5 * data_high_vol['darwdown'].std())
leverage

In [ ]:
leverage = 2

In [ ]:
taker_fee = 0.00055

In [ ]:
(leverage * (data_high_vol['return_pct'] - 2 * taker_fee) + 1).cumprod().plot()

In [ ]:
start_cash = 1000
risk = 1
leverage = 10
pf = Portfolio(start_cash=start_cash, max_pos_pct=1, max_leverage=100, maker_fee=0.0002, taker_fee=0.00055)

t = 1
for idx, candle in data_high_vol.iterrows():
    print(f'{t} / {data_high_vol.shape[0]}')
    t += 1
    entry_range = candle['candle_size_mean'] + std_multiple * candle['candle_size_std']
    
    start_date = idx - datetime.timedelta(minutes=tf_hours * 60)
    end_date = idx - datetime.timedelta(minutes=1)
    data_1m = Dataset().get_data(ticker='BTCUSDT', interval='1m', start_date=start_date.strftime('%Y-%m-%d %H:%M:%S'),
                                 end_date=end_date.strftime('%Y-%m-%d %H:%M:%S'))
    for i in data_1m.index:
        data_1m.loc[i, 'roll_high'] = data_1m.loc[:i]['high'].max()
        data_1m.loc[i, 'roll_low'] = data_1m.loc[:i]['low'].min()
    data_1m['roll_candle_size'] = data_1m['roll_high'] - data_1m['roll_low']
    rough_price = data_1m[data_1m['roll_candle_size'] > entry_range].iloc[0]['close']
    
    if rough_price > candle['open']:
        entry = candle['low'] + entry_range
        exit = candle['close']
        
        pf.open_trade(entry, sl=1000000, tp=0, date=idx, risk=risk, lev=leverage)
        if exit <= entry:
            pf.take_profit(tp=exit, date=idx, order_type='market')
        else:
            pf.stop_loss(sl=exit, date=idx)
        
        data_high_vol.loc[idx, 'entry'] = entry
        data_high_vol.loc[idx, 'exit'] = exit
        data_high_vol.loc[idx, 'return_pct'] = (entry - exit) / entry
        data_high_vol.loc[idx, 'darwdown'] = (candle['high'] - entry) / entry
    elif rough_price < candle['open']:
        entry = candle['high'] - entry_range
        exit = candle['close']
        
        pf.open_trade(entry, sl=0, tp=1000000, date=idx, risk=risk, lev=leverage)
        if exit >= entry:
            pf.take_profit(tp=exit, date=idx, order_type='market')
        else:
            pf.stop_loss(sl=exit, date=idx)  
        
        data_high_vol.loc[idx, 'entry'] = entry
        data_high_vol.loc[idx, 'exit'] = exit
        data_high_vol.loc[idx, 'return_pct'] = (exit - entry) / entry
        data_high_vol.loc[idx, 'darwdown'] = (entry - candle['low']) / entry
    else:
        data_high_vol.loc[idx, 'entry'] = np.nan
        data_high_vol.loc[idx, 'exit'] = np.nan
        data_high_vol.loc[idx, 'return_pct'] = 0
        data_high_vol.loc[idx, 'darwdown'] = 0

In [ ]:
pf.trade_ts